In [ ]:
import numpy as np
import pandas as pd 
import re, os, random
import torch
import cv2
from glob import glob
import matplotlib.pyplot as plt
from torchvision.models.detection.faster_rcnn import FasterRCNN
import albumentations as A
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import StratifiedKFold
from torch.utils.data.sampler import SequentialSampler, RandomSampler

from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from albumentations.pytorch.transforms import ToTensorV2
from albumentations import HorizontalFlip, VerticalFlip, RandomCrop

device = torch.device('cuda')

In [ ]:
HOME_DIR = '../input/global-wheat-detection'
SAVE_DIR = '/kaggle/working'
os.listdir(HOME_DIR)

In [ ]:
# read the csv

TRAIN_DIR = f'{HOME_DIR}/train/'
TEST_DIR = f'{HOME_DIR}/test/'

df = pd.read_csv('../input/boxes-repaired/train_clean.csv')
df.head()

In [ ]:
print('Train',len(os.listdir(TRAIN_DIR)))
print('Test',len(os.listdir(TEST_DIR)))
print('unique image id in df',len(df['image_id'].unique()))

In [ ]:
# make csv

def expand_bbox(x):
    r = np.array(re.findall("([0-9]+[.]?[0-9]*)", x))
    if len(r) == 0:
        r = [-1, -1, -1, -1]
    return r

df['x1'] = -1
df['y1'] = -1
df['x2'] = -1
df['y2'] = -1
df['area'] = -1

df[['x1','y1','x2','y2']] = np.stack(df['bbox'].apply(lambda x: expand_bbox(x)))
# df.drop(columns=['bbox'], inplace=True)
df.drop(columns=['x'], inplace=True)
df.drop(columns=['y'], inplace=True)
df.drop(columns=['w'], inplace=True)
df.drop(columns=['h'], inplace=True)

df = df.astype({'x1': 'float32', 'x2': 'float32', 
                'y1': 'float32', 'y2': 'float32'})

df['x2'] = df['x1'].values + df['x2'].values
df['y2'] = df['y1'].values + df['y2'].values

df['area'] = (df['y2'] - df['y1']) * (df['x2'] - df['x1'])

df.head()

In [ ]:
# 5 folds

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

df_folds = df[['image_id']].copy()
df_folds.loc[:, 'bbox_count'] = 1
df_folds = df_folds.groupby('image_id').count()
df_folds.loc[:, 'source'] = df[['image_id', 'source']].groupby('image_id').min()['source']
df_folds.loc[:, 'stratify_group'] = np.char.add(
    df_folds['source'].values.astype(str),
    df_folds['bbox_count'].apply(lambda x: f'_{x // 15}').values.astype(str)
)
df_folds.loc[:, 'fold'] = 0

for fold_number, (train_index, val_index) in enumerate(skf.split(X=df_folds.index, y=df_folds['stratify_group'])):
    df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number

In [ ]:
df_folds.head()

In [ ]:
class WheatDataset(Dataset):
       
    def __init__(self, dataframe, image_ids, data_dir, transforms=None):
        super().__init__()
        self.df = dataframe 
        self.image_list = image_ids
        self.image_dir = data_dir
        self.transforms = transforms
    
    def __len__(self):
        return len(self.image_list)
        
    def __getitem__(self, idx):
        
        image_id = self.image_list[idx]
        image_data = self.df.loc[self.df['image_id'] == image_id]
        boxes = torch.as_tensor(np.array(image_data[['x1','y1','x2','y2']]), 
                                dtype=torch.float32)
        area = torch.tensor(np.array(image_data['area']), dtype=torch.int64) 
        labels = torch.ones((image_data.shape[0],), dtype=torch.int64)
        iscrowd = torch.zeros((image_data.shape[0],), dtype=torch.uint8)
         
        target = {}
        target['area'] = area
        target['labels'] = labels
        target['iscrowd'] = iscrowd
        
        image = cv2.imread((self.image_dir + '/' + image_id + '.jpg'), cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        
        if self.transforms:
            
            image_transforms = {
                                'image': image,
                                'bboxes': boxes,
                                'labels': labels
                                 }
            
            image_transforms = self.transforms(**image_transforms)
            image = image_transforms['image']
            
            target['boxes'] = torch.as_tensor(image_transforms['bboxes'], dtype=torch.float32)
            target['boxes'] = target['boxes'].reshape(-1, 4) # recheck boxes dimension due to error
                 
        return image, target

In [ ]:
def get_train_transforms():
    return A.Compose([
          A.RandomSizedCrop(min_max_height=(800, 800), height=1024, width=1024, p=0.5),
           A.OneOf([
                A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, 
                                     val_shift_limit=0.2, p=0.9),
                A.RandomBrightnessContrast(brightness_limit=0.2, 
                                           contrast_limit=0.2, p=0.9),
              A.RandomGamma(p=0.5),
              A.GaussNoise(p=0.5,var_limit=(0, 0.2)),
              A.GaussianBlur(p=0.6),
              A.IAASharpen(p=0.5),
              A.RandomShadow(p=0.3)
            ],p=0.9),
        
        A.ToGray(p=0.2),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.3),
        A.RandomBrightnessContrast(p=0.3),
        A.Resize(height=1024, width=1024, p=1),
        ToTensorV2(p=1.0)
        
    ], bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        ))

def get_valid_transforms():
    return A.Compose([
        A.Resize(height=1024, width=1024, p=1),
        ToTensorV2(p=1.0)
    ])

def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
# new loader

NUM_WORKERS = 16
BATCH_SIZE = 8
fold_number = 0

train_dataset = WheatDataset(
    image_ids=df_folds[df_folds['fold'] != fold_number].index.values,
    data_dir=TRAIN_DIR,
    dataframe=df,
    transforms=get_train_transforms()
)
train_dataloader = DataLoader(train_dataset, 
                              batch_size=BATCH_SIZE,
                              sampler=RandomSampler(train_dataset),
                              num_workers=NUM_WORKERS,
                              collate_fn=collate_fn)

validation_dataset = WheatDataset(
    image_ids=df_folds[df_folds['fold'] == fold_number].index.values,
    data_dir=TRAIN_DIR,
    dataframe=df,
    transforms=get_valid_transforms()
)

valid_dataloader = DataLoader(validation_dataset, 
                              batch_size=4,
                              sampler=SequentialSampler(validation_dataset),
                              shuffle=False, 
                              num_workers=NUM_WORKERS,
                              collate_fn=collate_fn)

In [ ]:
def show_sample(sample):
    image, target = sample
    boxes = target['boxes'].cpu().numpy().astype(np.int32)

    numpy_image = image.permute(1,2,0).cpu().numpy()

    fig, ax = plt.subplots(1, 1, figsize=(16, 8))

    for box in boxes:
        cv2.rectangle(numpy_image, (box[0], box[1]), (box[2],  box[3]), (0, 1, 0), 2)

    ax.set_axis_off()
    ax.imshow(numpy_image);

In [ ]:
show_sample(train_dataset[30])

In [ ]:
len(train_dataset), len(validation_dataset)

In [ ]:
# train_dataset[0]

In [ ]:
backbone = resnet_fpn_backbone('resnet152', pretrained=True)
model = FasterRCNN(backbone, num_classes=2).to(device)

In [ ]:
# plot lr

params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.AdamW(params, lr=0.001, weight_decay=0.001)
# optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, weight_decay=0.0001)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)
# lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=6, eta_min=0, last_epoch=-1) # T_max is max iteration

EPOCHS = 25
lrs = []
for i in range(EPOCHS):
    # lr = er.get_last_lr()[0]
    lr1 = optimizer.param_groups[0]["lr"]
    lrs.append(lr1)
    # print(f'Step: {i},LR: {lr}')
    lr_scheduler.step()
plt.plot([i for i in range(EPOCHS)], lrs)

In [ ]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience.
       by https://github.com/Bjarten/early-stopping-pytorch"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
        
    def __call__(self, val_loss):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

In [ ]:
# metric

class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0
    
def save_model(path, best_loss):
    model.eval()
    torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': lr_scheduler.state_dict(),
            'best_summary_loss': best_loss,
            'epoch': epoch+1,
        }, path)

In [ ]:
%%time

model.train()
params = [p for p in model.parameters() if p.requires_grad]

early_stopping = EarlyStopping(patience=5, verbose=True)

optimizer = torch.optim.AdamW(params, lr=0.001, weight_decay=0.0001)
# optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0001)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)
# lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=6, eta_min=0, last_epoch=-1)
#lr_scheduler = None

num_epochs = 10
Train_losses = []
Valid_losses = []
loss_hist = Averager()
best_loss = 1.5
prev_epochs = 0
# grad_acc_steps = 16

for epoch in range(num_epochs):
    loss_hist.reset()

    for step, (images, targets) in enumerate(train_dataloader):
        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        loss_hist.send(loss_value)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    Train_losses.append(loss_hist.value)
    print(f"Epoch #{epoch+1} train_loss: {loss_hist.value}")
    loss_hist.reset()

    for images, targets in valid_dataloader:
        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        val_losses = sum(loss for loss in loss_dict.values())
        val_loss = val_losses.item()

        loss_hist.send(val_loss)

    Valid_losses.append(loss_hist.value)
    print(f"valid_loss: {loss_hist.value}")

    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

     # early stoping
    early_stopping(Valid_losses[-1])
    if early_stopping.early_stop:
        print("Early stopping")
        break    
        
    if best_loss > Valid_losses[-1]:

        best_loss = Valid_losses[-1]
        save_model(f'{SAVE_DIR}/best-checkpoint-{str(epoch).zfill(3)}epoch.bin', best_loss)
        # remove previous more than 3 saves
        for path in sorted(glob(f'{SAVE_DIR}/best-checkpoint-*epoch.bin'))[:-3]:
             os.remove(path)

In [ ]:
# last save 

save_model(f'{SAVE_DIR}/last-checkpoint-{str(num_epochs+prev_epochs).zfill(3)}epoch.bin', Valid_losses[-1])